In [1]:
from turret import Turret
from cybergear_motor_controller import CyberGearMotorController
import logging
import can

In [4]:
bus = can.Bus(channel="can0", interface="socketcan")
pitch_motor = CyberGearMotorController(bus=bus, motor_can_id=100)
yaw_motor = CyberGearMotorController(bus=bus, motor_can_id=101)
turret = Turret(yaw_motor, pitch_motor, dict())

SocketcanBus was not properly shut down


In [8]:
turret.zero_all()


In [7]:
# turret.disable()

3.0779078006744385 3.0779075622558594
0.6932108652254101 0.17453292519943295
0.5292728793283459 0.17453292519943295
0.1801190328260418 0.17453292519943295
